# 1. Вытаскиваем ссылки, которые накачали

In [ ]:
pip install fake_useragent

In [2]:
import requests
from bs4 import BeautifulSoup
from requests.exceptions import ConnectionError

# библиотека для создания юзерагентов
from fake_useragent import UserAgent

from tqdm import tqdm_notebook
import re
import pickle
import time
 
# модуль с мелкими функциями
from cian_info import *

In [ ]:
with open('hrefs_total.pickle', 'rb') as f:
    hrefs_total = pickle.load(f)
    
len(hrefs_total)

In [ ]:
type(hrefs_total), len(set(hrefs_total))

### Чтобы включать ТОР

In [ ]:
import socks
import socket
socks.set_default_proxy(socks.SOCKS5, "localhost", 9150)
socket.socket = socks.socksocket

In [ ]:
def checkIP():
    ip = requests.get('http://checkip.dyndns.org').content
    soup = BeautifulSoup(ip, 'html.parser')
    print(soup.find('body').text)

checkIP()

# 2. Качаем основную информацию по квартирам

Объединяем всё в одну функцию! 

In [ ]:
def infa_download(soup):
    infa_dict = {}
    infa_dict.update(get_general_information(soup))
    infa_dict.update(get_about(soup))
    infa_dict.update(get_price(soup))
    infa_dict.update(get_description(soup))
    infa_dict.update(get_square_info(soup))
    infa_dict.update(get_address(soup))
    infa_dict.update(get_metro(soup))
    infa_dict.update(get_title(soup))
    infa_dict.update(get_coord(soup))
    infa_dict.update(get_reinovation(soup))
    return infa_dict

Основной цикл: 

In [ ]:
all_infa = [ ] # для данных

In [ ]:
all_infa = data_new

In [ ]:
for url in tqdm_notebook(hrefs_total[34640:40000]):
    print(url)
    
    try:
        soup = get_soup(url)
        
    except ConnectionError:
        print('конэкшон ерор')
        time.sleep(5)
        soup = get_soup(url)
        
    # пока вылезает капча, отдыхаем
    try:
        while is_capcha_or_badip(soup):
            time.sleep(15)            # сплю N секунд
            soup = get_soup(url)     # внеочередная попытка пробится сквозь капчу 

        if soup.find("div", {"data-name": "OfferUnpublished"}):
            print('объявление снято')
            continue

        cur_infa = infa_download(soup)
        cur_infa['href'] = url
        all_infa.append(cur_infa)
    except AttributeError:
        continue

In [ ]:
hrefs_total.index(url) 
# чтобы понимать откуда продолжать скачку, если код упал

In [ ]:
len(all_infa)

In [ ]:
# осторожнее с именами файлов! случайно можно перезаписать готовый :( 
with open('data/info_30000_40000.pickle', 'wb') as f:
    pickle.dump(all_infa, f)

In [8]:
with open('data/info_30000_40000.pickle', 'rb') as f:
    data_new = pickle.load(f)

EOFError: Ran out of input

Поглазеть на данные: 

In [ ]:
import pandas as pd
df = pd.DataFrame(data_new)
df.tail()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()